<a href="https://colab.research.google.com/github/Srajan1122/Hyperspectral_Image_Classification/blob/srajan/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scipy import io
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
import sklearn.svm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading DataSet
### Dataset include image and a ground truth

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/Botswana/Botswana.mat')
for key, value in loaded_dataset.items():
  if isinstance(value, type(np.array([1]))):
    image = loaded_dataset[key]

In [0]:
ground_truth_1 = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/Botswana/Botswana_gt.mat')
for key, value in ground_truth_1.items():
  if isinstance(value, type(np.array([1]))):
    ground_truth = ground_truth_1[key]

### Class Labels

In [0]:
label_values = ["Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]

### Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

### Visualizing data in pandas

In [0]:
data = pd.DataFrame(final_output)

In [9]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145
0,3996,3952,3698,3457,3063,2914,2796,2538,2629,2613,2533,2517,2306,2149,2065,1994,1943,1840,1779,1726,1654,1519,1542,1478,1404,1600,2755,2806,3459,4137,4008,2741,4144,4268,4069,4020,3537,3335,3582,3868,...,450,385,316,240,157,98,149,142,106,91,107,158,164,163,166,184,196,181,169,156,159,134,135,144,159,142,138,145,138,110,104,114,89,89,77,73,60,53,54,0
1,3996,3952,3698,3457,3063,2914,2796,2538,2629,2613,2533,2517,2306,2149,2065,1994,1943,1840,1779,1726,1654,1519,1542,1478,1404,1600,2029,2172,2672,3195,3184,2182,3316,3402,3199,3147,2763,2517,2841,3098,...,427,371,301,226,145,92,128,134,92,85,114,140,149,158,159,166,175,160,151,137,141,133,135,132,143,134,123,133,122,117,109,103,83,93,80,69,59,54,57,0
2,4100,3923,3615,3451,2941,2886,2850,2581,2628,2616,2607,2479,2359,2118,2099,1946,1971,1885,1743,1774,1680,1566,1534,1494,1375,1603,2152,2277,2870,3474,3502,2441,3612,3677,3489,3410,2991,2866,3101,3280,...,365,324,270,199,131,70,95,101,76,75,98,105,124,134,138,135,140,123,118,111,112,123,119,110,114,103,95,107,98,113,102,85,67,89,79,55,52,51,56,0
3,3908,3889,3594,3343,2975,2813,2665,2445,2532,2489,2423,2328,2209,2039,1944,1871,1850,1700,1616,1564,1493,1398,1356,1345,1271,1463,2046,2243,2939,3522,3582,2506,3674,3793,3590,3536,3125,2942,3194,3397,...,370,324,263,200,143,68,100,103,85,74,86,111,126,139,147,125,121,125,120,106,116,117,106,110,118,109,107,106,108,103,94,75,61,73,64,50,57,48,45,0
4,4007,3926,3767,3445,3038,2945,2914,2588,2616,2624,2608,2509,2347,2176,2099,2002,1982,1889,1775,1687,1705,1531,1530,1454,1450,1631,2222,2426,3174,3859,3855,2711,4073,4139,3992,3948,3447,3264,3517,3747,...,380,327,259,198,142,67,102,116,94,68,82,116,127,147,154,118,106,122,120,106,119,106,97,112,119,122,119,113,111,94,88,63,57,62,51,49,56,42,41,0


### Droping the row if ground_truth value is zero

In [10]:
print("Percentage of column which will be droped",(data.size - data[data.iloc[:, -1] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.008595867208672087 %


In [0]:
zero_data = data.index[data.iloc[:, -1] == 0].tolist()
data = data[data.iloc[:, -1] != 0]

### Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

In [0]:
number_of_classes = len(np.unique(y))

### Feature Selection


In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

X = SelectKBest(chi2, k=int((image.shape[2]+1)*0.75)).fit_transform(X, y)

### OneHotEncoding the target column

In [0]:
# onehotencoder = OneHotEncoder() 
# y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray()

### Standardizing the feature columns

In [0]:
X = preprocessing.scale(X)

### Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60, random_state=25)

# Genetic Algorithm

In [18]:
!pip install pyeasyga

In [19]:
from pyeasyga import pyeasyga
import random
import numpy as np
from sklearn.metrics import accuracy_score
import time

def initilialize_population():
  C = list(np.logspace(-3, 3, 7))
  kernel = ['linear', 'poly', 'rbf', 'sigmoid']
  degree = list(x for x in range(3,8))
  gamma = ['scale', 'auto'] + list(np.logspace(-5, -1, 5))
  shrinking = [True, False]
  probability = [True, False]
  decision_function_shape =  ['ovo', 'ovr']
  
  parameters = []
  parameters.append(C)
  parameters.append(kernel)
  parameters.append(degree)
  parameters.append(gamma)
  parameters.append(shrinking)
  parameters.append(probability)
  parameters.append(decision_function_shape)

  return parameters
  

def create_individual(data):
  choice = []
  for i in range(len(data)):
    choice.append(random.choice(list(range(len(data[i])))))
  return choice
  

def fitness_function(individual, data):
  model = sklearn.svm.SVC(C=data[0][individual[0]],
                          kernel=data[1][individual[1]],
                          degree=data[2][individual[2]],
                          gamma=data[3][individual[3]],
                          coef0=0.0,
                          shrinking=data[4][individual[4]],
                          probability=data[5][individual[5]],
                          tol=0.001,
                          cache_size=200,
                          class_weight=None,
                          verbose=True,
                          max_iter=-1,
                          decision_function_shape=data[6][individual[6]],
                          break_ties=False,
                          random_state=None)
  model.fit(X_train, y_train)
  prediction = model.predict(X_test)
  score = accuracy_score(y_test, prediction)
  return score


data = initilialize_population()
ga = pyeasyga.GeneticAlgorithm(seed_data=data,
                               population_size=5,
                               generations=5,
                               crossover_probability=0.8,
                               mutation_probability=0.4,
                               elitism=True,
                               maximise_fitness=True)
ga.create_individual = create_individual
ga.fitness_function = fitness_function

start = time.time()
ga.run()
end = time.time()
print('Time taken ------>', end-start)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Time taken ------> 32.03387784957886


# Preparing SVM model

### Defining grid parameters

In [0]:
# SVM_GRID_PARAMS = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]},
#                    {'kernel': ['poly'], 'degree': [3], 'gamma': [1e-1, 1e-2, 1e-3]}]

# Best Parameters
# SVM_GRID_PARAMS = [{'kernel': ['rbf'], 'gamma': [1e-2],
#                     'C': [100]}]

### Defining the weight 

In [0]:
# class_weight = None

### Defining the model

Link to learn more about svm https://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use

In [22]:
best_parameters = ga.best_individual()[1]
for index,i in enumerate(data):
  print(i[best_parameters[index]])

print(ga.best_individual()[0])

0.1
linear
3
auto
True
False
ovo
0.9081580297588507


In [23]:
model = sklearn.svm.SVC(C=data[0][best_parameters[0]],
                        kernel=data[1][best_parameters[1]],
                        degree=data[2][best_parameters[2]],
                        gamma=data[3][best_parameters[3]],
                        coef0=0.0,
                        shrinking=data[4][best_parameters[4]],
                        probability=data[5][best_parameters[5]],
                        tol=0.001,
                        cache_size=200,
                        class_weight=None,
                        verbose=True,
                        max_iter=-1,
                        decision_function_shape=data[6][best_parameters[6]],
                        break_ties=False,
                        random_state=None)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
score = accuracy_score(y_test, prediction)

[LibSVM]

In [0]:
# model = sklearn.svm.SVC(C=1.0,
#                         kernel='rbf',
#                         degree=3,
#                         gamma='scale',
#                         coef0=0.0,
#                         shrinking=True,
#                         probability=False,
#                         tol=0.001,
#                         cache_size=200,
#                         class_weight=None,
#                         verbose=True,
#                         max_iter=-1,
#                         decision_function_shape='ovr',
#                         break_ties=False,
#                         random_state=None)
# model.fit(X_train, y_train)
# prediction = model.predict(X_test)
# score = accuracy_score(y_test, prediction)

### Applying the grid search

In [0]:
# model = sklearn.model_selection.GridSearchCV(model, SVM_GRID_PARAMS, verbose=5, n_jobs=4)

### Model fitting

In [0]:
# model.fit(X_train, y_train)

### Best parameters

In [0]:
# print("SVM best parameters : {}".format(model.best_params_))

### Accuracy of training set

In [28]:
model.score(X_train, y_train)

0.9399538106235565

### Cross Validation using kfold

In [0]:
# kfold = KFold(n_splits=2)

In [0]:
# result = cross_val_score(model, X_test, y_test, cv=kfold, scoring='accuracy')

In [0]:
# result.mean()

### Accuracy of testing set

In [0]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,cohen_kappa_score
prediction = model.predict(X_test)

In [33]:
accuracy_score(y_test, prediction)

0.9081580297588507

In [34]:
precision_score(y_test, prediction, average="weighted")

0.9097701686855948

In [35]:
recall_score(y_test, prediction, average="weighted")

0.9081580297588507

In [36]:
f1_score(y_test, prediction, average="weighted")

0.9082147537498777

In [37]:
cohen_kappa_score(y_test, prediction, labels=list(x for x in range(1,17)), weights=None, sample_weight=None)

0.9004786777312159

In [0]:
# print(classification_report(y_test, prediction, target_names=label_values))

In [39]:
!pip install spectral

In [40]:
145*145

21025

In [41]:
!pip install spectral

In [0]:
import spectral as sp
prediction = model.predict(X)

if not len(prediction.shape) == 2:
    y_test = [x + 1 for x in y_test]
    y_train = [x + 1 for x in y_train]

if len(prediction.shape) == 2:
    predicted_gt_1 = np.argmax(prediction, axis=1)
    predicted_gt_1_list = list(predicted_gt_1)
    predicted_gt_1_list = [x + 1 for x in predicted_gt_1_list]
else:
    predicted_gt_1_list = list(prediction)

for i in zero_data:
    predicted_gt_1_list.insert(i, 0)

predicted_gt_1_list = np.array(predicted_gt_1_list).reshape(image.shape[0], image.shape[1])
sp.save_rgb('predicted_gt.jpg', predicted_gt_1_list, colors=sp.spy_colors)